In [1]:
import collections
import os
import pickle
import sys
import time

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats

In [4]:
from snmachine import gps, snfeatures
from snmachine.utils.plasticc_pipeline import create_folder_structure, get_directories, load_dataset

In [5]:
%config Completer.use_jedi = False  

In [6]:
folder_path = 'C:\\Users\\Alcatraz\\Desktop\\Supernova\\snmachine-main\\snmachine\\example_data'

In [7]:
data_file_name = 'example_dataset_aug.pckl'

In [8]:
data_path = os.path.join(folder_path, data_file_name)
dataset = load_dataset(data_path)

Opening from binary pickle
Dataset loaded from pickle file as: <snmachine.sndata.PlasticcData object at 0x000001D89112F7F0>


In [9]:
analysis_name = data_file_name[:-5]

In [12]:
import os

def create_folder_structure(folder_path, analysis_name):

    for key, value in directories.items():
        os.makedirs(value, exist_ok=True)


folder_path = r'C:\Users\Alcatraz\Desktop\Supernova\snmachine-main\snmachine\example_data'

create_folder_structure(folder_path, analysis_name)


In [11]:
directories = get_directories(folder_path, analysis_name) 
path_saved_gps = directories['intermediate_files_directory']

In [13]:
t_min = 0
t_max = 278

gp_dim = 2
number_gp = 276
number_processes = 1

In [14]:
gps.compute_gps(dataset, number_gp=number_gp, t_min=t_min, t_max=t_max, 
                gp_dim=gp_dim, output_root=path_saved_gps, 
                number_processes=number_processes)

Performing Gaussian process regression.
Models fitted with the Gaussian Processes values.
Time taken for Gaussian process regression: 2.46s.


In [15]:
path_saved_wavelets = directories['intermediate_files_directory']

In [16]:
number_comps = 40
path_saved_reduced_wavelets = directories['features_directory']

In [17]:
wf = snfeatures.WaveletFeatures(output_root=path_saved_wavelets)


reduced_wavelet_features = wf.compute_reduced_features(
    dataset, number_comps=number_comps, 
    **{'wavelet_name': 'sym2', 'number_decomp_levels': 2,
       'path_save_eigendecomp': path_saved_reduced_wavelets})

The wavelet used is sym2.
Each passband is decomposed in 2 levels.
Performing wavelet decomposition.
Time taken for wavelet decomposition: 0.40s.
Performing eigendecomposition.
Time taken for eigendecomposition: 0.09s.
Dimensionality reduced feature space with 40 components.


In [18]:
path_saved_reduced_wavelets

'C:\\Users\\Alcatraz\\Desktop\\Supernova\\snmachine-main\\snmachine\\example_data\\example_dataset_aug\\wavelet_features'

In [19]:
wf.save_reduced_features(reduced_wavelet_features, path_saved_reduced_wavelets)

In [20]:
features = reduced_wavelet_features.copy() 

metadata = dataset.metadata
features['hostgal_photoz'] = metadata.hostgal_photoz.values.astype(float)
features['hostgal_photoz_err'] = metadata.hostgal_photoz_err.values.astype(float)

In [21]:
path_saved_features = directories['features_directory']

In [22]:
features.to_pickle(os.path.join(path_saved_features, 'features.pckl'))

data_labels = dataset.labels.astype(int)  
data_labels.to_pickle(os.path.join(path_saved_features, 'data_labels.pckl'))

In [23]:
saved_features = pd.read_pickle(os.path.join(path_saved_features, 'features.pckl')) 
saved_data_labels = pd.read_pickle(os.path.join(path_saved_features, 'data_labels.pckl'))

In [24]:
print(np.allclose(saved_features, features))
print(np.allclose(saved_data_labels, data_labels))

True
True
